# Rahib Khandaker PHW4

# Chapter 8 Exercise 2

- ## Delete from the dbo.Orders table orders that were placed before August 2014. Use the OUTPUT clause to return the orderid and orderdate values of the deleted orders
    
- ## Table involved: Sales.Order
    

### Desired Output:

| orderid | orderdate |
| --- | --- |
| 10248 | 2014-07-04 |
| 10249 | 2014-07-05 |
| 10250 | 2014-07-08 |
| 10251 | 2014-07-08 |
| 10252 | 2014-07-09 |
| 10253 | 2014-07-10 |
| 10254 | 2014-07-11 |
| 10255 | 2014-07-12 |
| 10256 | 2014-07-15 |
| 10257 | 2014-07-16 |
| 10258 | 2014-07-17 |
| 10259 | 2014-07-18 |
| 10260 | 2014-07-19 |
| 10261 | 2014-07-19 |
| 10262 | 2014-07-22 |
| 10263 | 2014-07-23 |
| 10264 | 2014-07-24 |
| 10265 | 2014-07-25 |
| 10266 | 2014-07-26 |
| 10267 | 2014-07-29 |
| 10268 | 2014-07-30 |
| 10269 | 2014-07-31 |

(22 row(s) affected)

In [ ]:
use TSQLV4
DELETE FROM dbo.Orders
OUTPUT deleted.orderid, deleted.orderdate
WHERE orderdate < '20140801';

# Proposition 1

Delete records of expired special deals that ended more than two years ago and output their details.

# Functional Specification

## Query Name
Delete Expired Special Deals

## Description
This query deletes entries from the `Sales.SpecialDeals` table for deals that ended more than two years prior to the current date. It uses the `OUTPUT` clause to return the `SpecialDealID` and `DealDescription` of each deleted record.

## Inputs
- **EndDate**: The date on which the special deal ended.

## Outputs
- **SpecialDealID**: The unique identifier of each special deal that is deleted.
- **DealDescription**: A brief description of the deleted deal.

## Steps
1. Identify records in the `Sales.SpecialDeals` table where `EndDate` is earlier than two years ago:
   - Calculate the date two years prior using `DATEADD(YEAR, -2, GETDATE())`.
   - Filter records where `EndDate` is earlier than the calculated date.
2. Delete all records that match this criteria.
3. Use the `OUTPUT` clause to capture the `SpecialDealID` and `DealDescription` of each deleted record.
4. Return the `SpecialDealID` and `DealDescription` for tracking purposes.

## Assumptions
- The data is available in the `Sales.SpecialDeals` table.
- The `EndDate` is stored in a valid date format.
- The current date can be determined using `GETDATE()`.

## Example Output

| SpecialDealID | DealDescription           |
|---------------|---------------------------|
| 101           | Winter Clearance Sale     |
| 102           | Summer Discount           |
| 103           | Year-End Special Offer    |
| 104           | Early Bird Holiday Deals  |


In [ ]:
USE WideWorldImporters
DELETE FROM Sales.SpecialDeals
OUTPUT DELETED.SpecialDealID, DELETED.DealDescription
WHERE EndDate < DATEADD(YEAR, -2, GETDATE())

# Proposition 2

Update the current stock levels by deducting quantities sold in the past month.

# Functional Specification

## Query Name
Update Stock Based on Recent Sales

## Description
This query updates the `QuantityOnHand` in the `Warehouse.StockItemHoldings` table by subtracting the total quantity sold from orders placed in the past month. It calculates the total quantity ordered per stock item and applies the deduction to the existing stock level.

## Inputs
- **StockItemID**: The unique identifier for each stock item.
- **OrderDate**: The date on which the order was placed.
- **Quantity**: The quantity ordered for each item.
- **QuantityOnHand**: The current stock level of each item.

## Outputs
- The `Warehouse.StockItemHoldings` table will have updated `QuantityOnHand` values based on recent sales.

## Steps
1. Identify all orders from the past month in the `Sales.Orders` table:
   - Filter for `OrderDate` within the last month using `DATEADD(MONTH, -1, GETDATE())`.
2. Join the `Sales.OrderLines` table on `OrderID` to retrieve the quantities ordered.
3. Join the `Warehouse.StockItemHoldings` table on `StockItemID` to match stock items.
4. For each `StockItemID`, calculate the total quantity sold in the last month and subtract it from the `QuantityOnHand`.
5. Group by `StockItemID` to ensure correct aggregation of sold quantities per item.
6. Update the `QuantityOnHand` in `Warehouse.StockItemHoldings` for each `StockItemID` based on these calculations.

## Assumptions
- The data is available in the `Warehouse.StockItemHoldings`, `Sales.OrderLines`, and `Sales.Orders` tables.
- The `OrderDate` and `QuantityOnHand` values are in valid date and numeric formats, respectively.
- There are no negative values for `QuantityOnHand` after updating.

## Example Output

The `Warehouse.StockItemHoldings` table after updating might have entries as follows:

| StockItemID | QuantityOnHand |
|-------------|----------------|
| 101         | 250            |
| 102         | 125            |
| 103         | 300            |
| 104         | 475            |


In [ ]:
use WideWorldImporters
SELECT *
from Warehouse.StockItemHoldings

UPDATE sh
SET sh.QuantityOnHand = sh.QuantityOnHand - sub.TotalQuantity
FROM Warehouse.StockItemHoldings sh
    INNER JOIN (
    SELECT ol.StockItemID, SUM(ol.Quantity) AS TotalQuantity
    FROM Sales.OrderLines ol
        INNER JOIN Sales.Orders o ON ol.OrderID = o.OrderID
    WHERE o.OrderDate >= DATEADD(MONTH, -1, (SELECT max(PickingCompletedWhen)
    from Sales.OrderLines))
    GROUP BY ol.StockItemID
) sub ON sh.StockItemID = sub.StockItemID;


SELECT *
from Warehouse.StockItemHoldings


# Proposition 3

Insert new color entries into the `Warehouse.Colors` table from the `Warehouse.Colors_Archive` table.

# Functional Specification

## Query Name
Insert Colors from Archive

## Description
This query inserts new color records into the `Warehouse.Colors` table by selecting color names and the user who last edited from the `Warehouse.Colors_Archive` table. The `ColorID` for each new entry is generated by taking the maximum existing `ColorID` from the `Warehouse.Colors` table and incrementing it by one.

## Inputs
- **ColorID**: The unique identifier for each color (automatically generated).
- **ColorName**: The name of the color being inserted.
- **LastEditedBy**: The identifier of the user who last edited the color.

## Outputs
- The `Warehouse.Colors` table will contain new entries with unique `ColorID`, `ColorName`, and `LastEditedBy` based on the records from `Warehouse.Colors_Archive`.

## Steps
1. Identify the maximum `ColorID` from the `Warehouse.Colors` table:
   - Use `SELECT MAX(ColorID) FROM Warehouse.Colors`.
2. Increment the maximum `ColorID` by one to generate a new unique identifier.
3. Select `ColorName` and `LastEditedBy` from the `Warehouse.Colors_Archive` table.
4. Insert the new records into the `Warehouse.Colors` table with the generated `ColorID`, along with the selected `ColorName` and `LastEditedBy`.

## Assumptions
- The `Warehouse.Colors_Archive` table contains valid `ColorName` and `LastEditedBy` values.
- There are no duplicate `ColorName` entries in the `Warehouse.Colors` table after the insertion.
- The database allows for sequential `ColorID` assignment without conflicts.

## Example Output

After executing the query, the `Warehouse.Colors` table might have entries like the following:

| ColorID | ColorName    | LastEditedBy |
|---------|--------------|---------------|
| 1       | Red          | UserA        |
| 2       | Blue         | UserB        |
| 3       | Green        | UserC        |
| 4       | Yellow       | UserD        |
| 5       | Purple       | UserE        |
| 6       | Orange       | UserF        |


In [ ]:
use WideWorldImporters

INSERT INTO Warehouse.colors
    (ColorID, ColorName,LastEditedBy)
SELECT (SELECT MAX(ColorID)
    from Warehouse.Colors)+1, ColorName, LastEditedBy
from Warehouse.Colors_Archive

SELECT * from Warehouse.Colors
ORDER by ColorID DESC

# Proposition 4

Synchronize the `application.cities` table with the `application.cities_archive` table by updating existing records and inserting new ones as necessary.

# Functional Specification

## Query Name
Merge Cities from Archive

## Description
This query merges records from the `application.cities_archive` table into the `application.cities` table. If a record with a matching `CityID` exists in the target table, it is updated with the corresponding values from the source table. If no match is found, a new record is inserted into the target table.

## Inputs
- **CityID**: The unique identifier for each city.
- **CityName**: The name of the city.
- **StateProvinceID**: The identifier for the state or province.
- **Location**: The geographical location of the city.
- **LatestRecordedPopulation**: The most recent population count for the city.
- **LastEditedBy**: The identifier of the user who last edited the record.

## Outputs
- The `application.cities` table will be updated with the latest data from `application.cities_archive`, reflecting both updates and new entries.

## Steps
1. Identify records in `application.cities_archive` that need to be merged into `application.cities`.
2. Use `MERGE` to compare records based on `CityID`.
3. For records with matching `CityID`:
   - Update the corresponding fields in the `application.cities` table.
4. For records without a matching `CityID`:
   - Insert the new records from `application.cities_archive` into `application.cities`.

## Assumptions
- Both `application.cities` and `application.cities_archive` tables have valid and consistent `CityID` values.
- The columns being updated and inserted are compatible in data type and format.
- The `application.cities` table is structured to accept new records as defined.

## Example Output

After executing the query, the `application.cities` table might have entries like the following:

| CityID | CityName      | StateProvinceID | Location       | LatestRecordedPopulation | LastEditedBy |
|--------|---------------|------------------|----------------|--------------------------|---------------|
| 1      | New York      | 101              | 40.7128 N, 74.0060 W | 8419600                  | UserA        |
| 2      | Los Angeles   | 102              | 34.0522 N, 118.2437 W | 3980400                  | UserB        |
| 3      | Chicago       | 103              | 41.8781 N, 87.6298 W | 2716000                  | UserC        |
| 4      | Houston       | 104              | 29.7604 N, 95.3698 W | 2328000                  | UserD        |
| 5      | Phoenix       | 105              | 33.4484 N, 112.0740 W | 1680992                  | UserE        |


In [ ]:
use WideWorldImporters

SELECT * from Application.Cities c
join Application.Cities_Archive ca on ca.CityID =c.CityID
order by c.CityID DESC


MERGE INTO application.cities AS target
USING application.cities_archive AS source
ON target.CityID = source.CityID  -- Match on CityID
WHEN MATCHED THEN
    UPDATE SET
        target.CityName = source.CityName,
        target.StateProvinceID = source.StateProvinceID,
        target.Location = source.Location,
        target.LatestRecordedPopulation = source.LatestRecordedPopulation,
        target.LastEditedBy = source.LastEditedBy
WHEN NOT MATCHED THEN
    INSERT (CityID, CityName, StateProvinceID, Location, LatestRecordedPopulation, LastEditedBy)
    VALUES (source.CityID, source.CityName, source.StateProvinceID, source.Location, source.LatestRecordedPopulation, source.LastEditedBy);


SELECT * from Application.Cities c
join Application.Cities_Archive ca on ca.CityID =c.CityID
order by c.CityID DESC


# Proposition 5

Update the credit limit for customers in a specific category by increasing it by 10%.

# Functional Specification

## Query Name
Update Credit Limit for Customers

## Description
This query updates the `CreditLimit` for all customers in the `Sales.Customers` table who belong to a specified category. The credit limit is increased by 10%, and the updated information is outputted for verification.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **CustomerName**: The name of the customer.
- **CreditLimit**: The current credit limit for each customer.

## Outputs
- The query will output the `CustomerID`, `CustomerName`, and the updated `CreditLimit` for each customer whose credit limit has been modified.

## Steps
1. Identify customers in the `Sales.Customers` table where `CustomerCategoryID` equals 3.
2. Increase the `CreditLimit` for these customers by multiplying it by 1.10.
3. Use the `OUTPUT` clause to return the `CustomerID`, `CustomerName`, and new `CreditLimit` for verification.

## Assumptions
- The `Sales.Customers` table contains valid `CustomerCategoryID` values.
- The `CreditLimit` values are numeric and can be increased without causing any constraints or business rules violations.

## Example Output

After executing the query, the output might look like this:

| CustomerID | CustomerName   | CreditLimit |
|------------|----------------|-------------|
| 101        | Acme Corp      | 11000.00    |
| 102        | Beta LLC       | 22000.00    |
| 103        | Gamma Inc      | 33000.00    |
| 104        | Delta Co       | 44000.00    |


In [ ]:
USE WideWorldImporters

UPDATE Sales.Customers
SET CreditLimit = CreditLimit * 1.10  -- Increase credit limit by 10%
OUTPUT inserted.CustomerID, inserted.CustomerName, inserted.CreditLimit
WHERE CustomerCategoryID = 3;  -- Example category


# Proposition 6

Delete orders from the `Sales.Orders` table that were placed before a specified date.(2013-02-01)

# Functional Specification

## Query Name
Delete Old Orders

## Description
This query deletes records from the `Sales.Orders` table for all orders that have an `OrderDate` earlier than February 1, 2013. This helps in managing data retention and ensuring that only recent orders are kept in the database.

## Inputs
- **OrderDate**: The date on which the order was placed.

## Outputs
- The `Sales.Orders` table will have all orders placed before the specified date removed.

## Steps
1. Identify all records in the `Sales.Orders` table where `OrderDate` is less than `'2013-02-01'`.
2. Execute the `DELETE` statement to remove these records from the table.

## Assumptions
- The `Sales.Orders` table contains valid `OrderDate` values in a recognizable date format.
- Deleting these records will not violate any foreign key constraints or business rules.

## Example Output

After executing the query, the `Sales.Orders` table will no longer contain entries with `OrderDate` before February 1, 2013. 

For example, the deleted records may have included entries like the following:

| OrderID | OrderDate   | CustomerID |
|---------|-------------|------------|
| 1001    | 2013-01-15  | 201        |
| 1002    | 2012-12-30  | 202        |
| 1003    | 2013-01-25  | 203        |


In [ ]:
USE WideWorldImporters

DELETE FROM Sales.Orders
WHERE OrderDate < '2013-01-01'; 


# Proposition 7

Update the standard discount percentage for customers in certain categories based on their average credit limit.

# Functional Specification

## Query Name
Update Standard Discount Percentage for Customers

## Description
This query updates the `StandardDiscountPercentage` for customers in the `Sales.Customers` table whose `CustomerCategoryID` meets a specific condition. The discount percentage is set to 8% for categories where the average credit limit exceeds 2800.

## Inputs
- **CustomerCategoryID**: The unique identifier for each customer category.
- **StandardDiscountPercentage**: The discount percentage to be applied.

## Outputs
- The `Sales.Customers` table will have updated `StandardDiscountPercentage` values for eligible customer categories.

## Steps
1. Identify `CustomerCategoryID` values from the `Sales.Customers` table where the average `CreditLimit` is greater than 2800.
   - This is done by grouping the records by `CustomerCategoryID` and applying the `HAVING` clause.
2. Update the `StandardDiscountPercentage` for customers in these identified categories, setting it to 0.08 (or 8%).

## Assumptions
- The `Sales.Customers` table contains valid `CreditLimit` values.
- The `CustomerCategoryID` and `StandardDiscountPercentage` fields can be updated without conflicts.
- There are customers in the `Sales.Customers` table that satisfy the average credit limit condition.

## Example Output

After executing the query, the `Sales.Customers` table may have entries where the `StandardDiscountPercentage` is updated as follows:

| CustomerID | CustomerCategoryID | StandardDiscountPercentage |
|------------|--------------------|----------------------------|
| 201        | 1                  | 0.08                       |
| 202        | 2                  | 0.08                       |
| 203        | 3                  | 0.05                       |
| 204        | 4                  | 0.08                       |


In [ ]:
use WideWorldImporters

UPDATE Sales.Customers
SET StandardDiscountPercentage = 0.08  -- Set to 8%
WHERE CustomerCategoryID IN (
    SELECT CustomerCategoryID
    FROM Sales.Customers
    GROUP BY CustomerCategoryID
    HAVING AVG(CreditLimit) > 2800  -- Average credit limit condition
);


# Proposition 9

Create a summary table of total spending for each customer based on their orders.

# Functional Specification

## Query Name
Generate Customer Order Summary

## Description
This query generates a summary of total spending for each customer by calculating the sum of the product of `Quantity` and `UnitPrice` from the `Sales.OrderLines` table. The results are inserted into a new table called `Sales.CustomerOrderSummary`.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **CustomerName**: The name of the customer.
- **Quantity**: The quantity of items ordered.
- **UnitPrice**: The price per unit of the ordered items.

## Outputs
- A new table `Sales.CustomerOrderSummary` containing the total amount spent by each customer.

## Steps
1. Select the `CustomerID` and `CustomerName` from the `Sales.Customers` table.
2. Join the `Sales.Orders` table on `CustomerID` to link customers to their orders.
3. Join the `Sales.OrderLines` table on `OrderID` to get details of the items ordered.
4. Calculate the total spent by multiplying `Quantity` and `UnitPrice`, and summing the results.
5. Group the results by `CustomerID` and `CustomerName`.
6. Insert the summarized data into the `Sales.CustomerOrderSummary` table.

## Assumptions
- The `Sales.Customers`, `Sales.Orders`, and `Sales.OrderLines` tables contain valid and consistent data.
- The `Sales.CustomerOrderSummary` table does not already exist or can be created without conflicts.

## Example Output

After executing the query, the `Sales.CustomerOrderSummary` table might have entries like the following:

| CustomerID | CustomerName   | TotalSpent |
|------------|----------------|------------|
| 201        | Acme Corp      | 15000.00   |
| 202        | Beta LLC       | 23000.00   |
| 203        | Gamma Inc      | 5000.00    |
| 204        | Delta Co       | 12000.00   |


In [ ]:
USE WideWorldImporters

SELECT C.CustomerID, C.CustomerName, SUM(Quantity*UnitPrice) AS TotalSpent
INTO Sales.CustomerOrderSummary
FROM Sales.Customers C
JOIN Sales.Orders O ON C.CustomerID = O.CustomerID
join Sales.OrderLines Ol on ol.OrderID = o.OrderID 
GROUP BY C.CustomerID, C.CustomerName;


# Proposition 10

Increase the credit limit for customers whose total orders exceed $100,000.

# Functional Specification

## Query Name
Update Credit Limit for High-Value Customers

## Description
This query updates the `CreditLimit` for customers in the `Sales.Customers` table by adding $5,000 to their existing credit limit. The update is applied only to those customers whose total order amount exceeds $100,000.

## Inputs
- **CustomerID**: The unique identifier for each customer.
- **CreditLimit**: The current credit limit for each customer.

## Outputs
- The `CreditLimit` for eligible customers in the `Sales.Customers` table will be increased by $5,000.

## Steps
1. Identify `CustomerID` values from the `Sales.Customers` table whose total order amount exceeds $100,000.
   - This is done by joining the `Sales.Orders` and `Sales.OrderLines` tables and grouping the results by `CustomerID`.
   - Apply the `HAVING` clause to filter customers based on the total spending (`SUM(Quantity * UnitPrice)`).
2. Update the `CreditLimit` for these identified customers, adding $5,000 to the current value.

## Assumptions
- The `Sales.Customers`, `Sales.Orders`, and `Sales.OrderLines` tables contain valid and consistent data.
- The `CreditLimit` values are numeric and can be increased without exceeding any business rules or constraints.
- There are customers who meet the criteria for the credit limit increase.

## Example Output

After executing the query, the `Sales.Customers` table might reflect changes in `CreditLimit` for eligible customers as follows:

| CustomerID | CurrentCreditLimit | UpdatedCreditLimit |
|------------|--------------------|---------------------|
| 201        | 15000              | 20000               |
| 202        | 25000              | 30000               |
| 203        | 8000               | 8000                |  -- Not eligible
| 204        | 30000              | 35000               |


In [ ]:
USE WideWorldImporters

UPDATE Sales.Customers
SET CreditLimit = CreditLimit + 5000  -- Increase credit limit
WHERE CustomerID IN (
    SELECT C.CustomerID
    FROM Sales.Customers C
    JOIN Sales.Orders O ON C.CustomerID = O.CustomerID
    join Sales.OrderLines Ol on ol.OrderID = o.OrderID 
    GROUP BY C.CustomerID
    HAVING SUM(Quantity*UnitPrice) > 100000  -- Customers with total orders over $100,000
);
